In [1]:
seed=197

In [2]:
# importing required libraries
import torch.nn as nn
import torch
import torch.nn.functional as F
import math,copy,re
import warnings
import pandas as pd
import numpy as np
import random
import pickle
import torch.nn.init as init
from torchviz import make_dot
import plotly.graph_objects as go
import pickle
# import seaborn as sns
# import torchtext
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import torch.nn.functional as F
import torch.optim as optim
from IPython.display import display
import matplotlib.pyplot as plt
import os 
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

warnings.simplefilter("ignore")
print(torch.__version__)

1.13.1+cu117


In [3]:
# Check if CUDA is available
if torch.cuda.is_available():
    # CUDA is available, so PyTorch can use a GPU
    device = torch.device("cuda")
    print("PyTorch is using GPU.")
else:
    # CUDA is not available, PyTorch will use CPU
    device = torch.device("cpu")
    print("PyTorch is using CPU.")

# Create a tensor and move it to the device
tensor = torch.randn(3, 4).to(device)

# Check the device of the tensor
print("Tensor device:", tensor.device)

PyTorch is using GPU.
Tensor device: cuda:0


In [4]:
def reset_random_seeds(seed):
    os.environ['PYTHONHASHSEED']=str(seed)
#     tf.random.set_seed(seed)
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

In [5]:
# pickle_file_path = '/media/terminator/Data/NatureData/Auto Ten Minutes.pkl'
pickle_file_path = '/media/terminator/NatureData/Thirty Minutes.pkl'

In [6]:
# Load data from the pickle file
with open(pickle_file_path, "rb") as f:
    loaded_data_dict = pickle.load(f)


In [7]:
# Retrieve data and labels
X_train = loaded_data_dict["X_train"]
y_train = loaded_data_dict["y_train"]
X_test = loaded_data_dict["X_test"]
y_test = loaded_data_dict["y_test"]

In [8]:
X_train.shape,y_train.shape

(torch.Size([23328, 144, 13]), torch.Size([23328, 3]))

In [9]:
features=13

In [10]:
def reconstructData(packets):
    # Take the first packet
    first_packet = packets[0]
    
    # Concatenate the last index of every packet
    reconstructed_data = np.concatenate([packet[-1:] for packet in packets[1:]])
    
    # Combine the first packet with the concatenated data
    reconstructed_data = np.concatenate([first_packet, reconstructed_data])
    
    return reconstructed_data

In [11]:
Valid,Test = X_test[:100000],X_test[100000:]
Valid_label,Test_label = y_test[:100000],y_test[100000:]

In [12]:
num_epochs = 500
# Define batch size
batch_size = 256
# Create DataLoader for training set
train_dataset = torch.utils.data.TensorDataset(X_train.float(), y_train.float())
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
valid_dataset = torch.utils.data.TensorDataset(Valid.float(), Valid_label.float())
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
test_dataset = torch.utils.data.TensorDataset(Test.float(), Test_label.float())
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [13]:
print(X_train.dtype)
print(y_train.dtype)

torch.float64
torch.float64


In [14]:
X_train.shape, y_train.shape

(torch.Size([23328, 144, 13]), torch.Size([23328, 3]))

In [15]:
import torch
import torch.nn as nn

class TimeSeriesEncoder(nn.Module):
    def __init__(self, input_channels):
        super(TimeSeriesEncoder, self).__init__()
        
        self.encoder = nn.Sequential(
            nn.Conv1d(input_channels, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=1),  # Reduce sequence length to 36 / 4 = 9
            nn.Conv1d(128, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=1)  # Reduce sequence length to 9 / 2 = 4.5 (rounded to 4)
        )
        self.classifier = nn.Sequential(
            nn.Linear(64*142, 1024),
            nn.ReLU(), 
            nn.Linear(1024, 3), 
            nn.Softmax(dim=1)  
        )
        
    
    def forward(self, x):
        x = x.permute(0,2,1)
        x = self.encoder(x)
#         print(x.shape)
        x = torch.reshape(x,(x.shape[0],-1))
        x = self.classifier(x)
#         x = x.permute(0,2,1)
        return x

# Create the autoencoder model
input_channels = 13  # Number of input features
# encoder = TimeSeriesEncoder(input_channels).to(device)





In [16]:
train = 1
avg = 'weighted'
seq_length = 144
num_epochs = 5000
reset_random_seeds(seed)
src_vocab_size = features+5
target_vocab_size = features
num_layers = 2
latent_dim = 8
embed_dim = 128
n_heads = 8
expansion_factor = 2

Cf = TimeSeriesEncoder(input_channels).to(device)

criterion = nn.CrossEntropyLoss()
optimizer_cf = optim.Adam(Cf.parameters(), lr=0.00001)

# Define early stopping parameters
patience = 100
best_loss = 0
epochs_without_improvement = 0

if train:
    # Step 4: Training loop
    for epoch in range(num_epochs):
        running_loss = 0.0
        latents = []
        PR, PL = [], []
        train_input,train_output = [], []

        # Iterate over mini-batches
        for i, (inputs, labels) in enumerate(train_loader):
            
            probability = 0.0
            coin_toss = random.random()  

            if coin_toss < probability:
                continue
            
            # Clear gradients
            optimizer_cf.zero_grad()
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            # Forward pass
            
            prediction = Cf(inputs)
            
            
            # Calculate loss
            loss = criterion(prediction, labels)
            loss = loss.float()

            # Backpropagation
            loss.backward()
            
            optimizer_cf.step()
            
            # Update running loss
            running_loss += loss.item()
            
            
            PR.append(prediction)
            PL.append(labels)
            

        # Calculate average loss per epoch
        PR = torch.cat(PR, dim=0)
        PL = torch.cat(PL, dim=0)
        _, predicted = torch.max(PR.data, 1)
        _, lbl = torch.max(PL.data, 1)
        total = lbl.size(0)
        correct = (predicted == lbl).sum().item()

        accuracy = 100 * correct / total
        print(f'Train Accuracy: {accuracy:.2f}%')
        conf_matrix = confusion_matrix(lbl.cpu().detach().numpy(),predicted.cpu().detach().numpy())
        print('Confusion Matrix:')
        print(conf_matrix)
        # Calculate average loss per epoch
        epoch_loss = running_loss / len(train_loader)
        print(f"Epoch: {epoch+1}, Train Loss: {round(epoch_loss,6)}")
        
        if epoch%1==0:
            valid_latents = []
            VR,VL = [], []
            valid_loss = 0.0
            for j, (inp, lab) in enumerate(valid_loader):
                probability = 0.0
                coin_toss = random.random()  

                if coin_toss < probability:
                    continue
                inp = inp.to(device)
                lab = lab.to(device)
                
                with torch.no_grad():
                    # Forward pass
                    outp = Cf(inp)
                    
                VR.append(outp)
                VL.append(lab)
            
            VR = torch.cat(VR, dim=0)
            VL = torch.cat(VL, dim=0)
            _, vpre = torch.max(VR.data, 1)
            _, vlbl = torch.max(VL.data, 1)
            total = vlbl.size(0)
            correct = (vpre == vlbl).sum().item()

            vaccuracy = 100 * correct / total
            print('-----------------------------------------')
            print(f'Valid Accuracy: {vaccuracy:.2f}%')
            lb = vlbl.cpu().detach().numpy()
            pre = vpre.cpu().detach().numpy()
            conf_matrix = confusion_matrix(lb,pre)
            precision = precision_score(lb, pre,average=avg)
            recall = recall_score(lb, pre,average=avg)
            f1 = f1_score(lb, pre,average=avg)
            print('Confusion Matrix:')
            print(conf_matrix)
            print()
            print("Precision:", precision)
            print("Recall:", recall)
            print("F1 Score:", f1)
            print(epochs_without_improvement, best_loss)
            print('-----------------------------------------')
            print()
            print()
            
                
                    
        # Check for early stopping and save the best weights
        th = (conf_matrix[0][0]> conf_matrix[0][1]+conf_matrix[0][2] and conf_matrix[1][1]>conf_matrix[1][0]+conf_matrix[1][2] and conf_matrix[2][2]>conf_matrix[2][0]+conf_matrix[2][1])
        if th and vaccuracy > best_loss:
            best_loss = vaccuracy
            epochs_without_improvement = 0
            # Save the model weights
            torch.save(Cf.state_dict(), 'cnn30.pt')
            print('##############################################')
            print('saved', best_loss)
            print('##############################################')
            print()
            print()
            
        else:
            epochs_without_improvement += 1
            if epochs_without_improvement >= patience:
                print("Early stopping triggered. Training stopped.")
                break

# Recover the best weights
Cf.load_state_dict(torch.load('cnn30.pt'))


Train Accuracy: 48.70%
Confusion Matrix:
[[4328   11 3437]
 [2149   20 5607]
 [ 751   13 7012]]
Epoch: 1, Train Loss: 1.078529
-----------------------------------------
Valid Accuracy: 51.84%
Confusion Matrix:
[[51102     0 45380]
 [  666     0  2083]
 [   26     0   743]]

Precision: 0.9520479318397514
Recall: 0.51845
F1 Score: 0.6652665335517525
0 0
-----------------------------------------


Train Accuracy: 50.87%
Confusion Matrix:
[[4329    3 3444]
 [1834   15 5927]
 [ 253    0 7523]]
Epoch: 2, Train Loss: 1.024517
-----------------------------------------
Valid Accuracy: 52.23%
Confusion Matrix:
[[51484    25 44973]
 [  684     1  2064]
 [   26     0   743]]

Precision: 0.9528723513531063
Recall: 0.52228
F1 Score: 0.6684570905748074
1 0
-----------------------------------------


Train Accuracy: 53.91%
Confusion Matrix:
[[4331  289 3156]
 [1831  732 5213]
 [ 254    8 7514]]
Epoch: 3, Train Loss: 0.990923
-----------------------------------------
Valid Accuracy: 52.84%
Confusion Ma

Train Accuracy: 74.72%
Confusion Matrix:
[[5308 2010  458]
 [2409 4698  669]
 [ 103  248 7425]]
Epoch: 19, Train Loss: 0.802987
-----------------------------------------
Valid Accuracy: 67.86%
Confusion Matrix:
[[65519 24749  6214]
 [  882  1606   261]
 [   11    23   735]]

Precision: 0.9543043131347378
Recall: 0.6786
F1 Score: 0.7805853744264426
0 67.715
-----------------------------------------


##############################################
saved 67.86
##############################################


Train Accuracy: 74.90%
Confusion Matrix:
[[5340 1986  450]
 [2407 4706  663]
 [ 102  248 7426]]
Epoch: 20, Train Loss: 0.799173
-----------------------------------------
Valid Accuracy: 67.95%
Confusion Matrix:
[[65613 24700  6169]
 [  885  1603   261]
 [   11    23   735]]

Precision: 0.9542848110608722
Recall: 0.67951
F1 Score: 0.7812441202420065
0 67.86
-----------------------------------------


##############################################
saved 67.951
##########################

Train Accuracy: 79.63%
Confusion Matrix:
[[5334 2082  360]
 [1516 5823  437]
 [ 107  250 7419]]
Epoch: 39, Train Loss: 0.758911
-----------------------------------------
Valid Accuracy: 67.15%
Confusion Matrix:
[[64384 27501  4597]
 [  546  2033   170]
 [   11    24   734]]

Precision: 0.9594615651658583
Recall: 0.67151
F1 Score: 0.7749023530945374
18 67.951
-----------------------------------------


Train Accuracy: 79.77%
Confusion Matrix:
[[5338 2081  357]
 [1490 5854  432]
 [ 108  251 7417]]
Epoch: 40, Train Loss: 0.757745
-----------------------------------------
Valid Accuracy: 67.18%
Confusion Matrix:
[[64410 27514  4558]
 [  546  2040   163]
 [   11    24   734]]

Precision: 0.9594787568583902
Recall: 0.67184
F1 Score: 0.7751122324971956
19 67.951
-----------------------------------------


Train Accuracy: 79.93%
Confusion Matrix:
[[5338 2081  357]
 [1472 5884  420]
 [ 110  243 7423]]
Epoch: 41, Train Loss: 0.756625
-----------------------------------------
Valid Accuracy: 67.1

Train Accuracy: 81.67%
Confusion Matrix:
[[5450 2008  318]
 [1255 6136  385]
 [ 129  180 7467]]
Epoch: 59, Train Loss: 0.741217
-----------------------------------------
Valid Accuracy: 68.04%
Confusion Matrix:
[[65151 27206  4125]
 [  440  2153   156]
 [   11    20   738]]

Precision: 0.961332379092295
Recall: 0.68042
F1 Score: 0.781280121007476
0 67.98
-----------------------------------------


##############################################
saved 68.042
##############################################


Train Accuracy: 81.83%
Confusion Matrix:
[[5462 1997  317]
 [1245 6151  380]
 [ 131  169 7476]]
Epoch: 60, Train Loss: 0.740348
-----------------------------------------
Valid Accuracy: 68.09%
Confusion Matrix:
[[65176 27191  4115]
 [  429  2175   145]
 [   11    19   739]]

Precision: 0.9615217708081711
Recall: 0.6809
F1 Score: 0.7815571545996275
0 68.042
-----------------------------------------


##############################################
saved 68.09
############################

-----------------------------------------
Valid Accuracy: 69.62%
Confusion Matrix:
[[66681 25775  4026]
 [  431  2198   120]
 [   11    19   739]]

Precision: 0.9617886520525872
Recall: 0.69618
F1 Score: 0.7924107212422351
0 69.436
-----------------------------------------


##############################################
saved 69.618
##############################################


Train Accuracy: 83.29%
Confusion Matrix:
[[5661 1821  294]
 [1186 6275  315]
 [ 173  110 7493]]
Epoch: 76, Train Loss: 0.728704
-----------------------------------------
Valid Accuracy: 69.75%
Confusion Matrix:
[[66808 25644  4030]
 [  431  2198   120]
 [   11    19   739]]

Precision: 0.9618098477092543
Recall: 0.69745
F1 Score: 0.7933128329982782
0 69.618
-----------------------------------------


##############################################
saved 69.745
##############################################


Train Accuracy: 83.36%
Confusion Matrix:
[[5674 1809  293]
 [1181 6280  315]
 [ 176  107 7493]]
Epoch:

...

Train Accuracy: 92.26%
Confusion Matrix:
[[7412  200  164]
 [ 978 6575  223]
 [ 181   60 7535]]
Epoch: 995, Train Loss: 0.628202
-----------------------------------------
Valid Accuracy: 91.61%
Confusion Matrix:
[[88555  5362  2565]
 [  355  2306    88]
 [   24     0   745]]

Precision: 0.9706614180298515
Recall: 0.91606
F1 Score: 0.9365199646502017
19 92.743
-----------------------------------------


Train Accuracy: 92.26%
Confusion Matrix:
[[7412  198  166]
 [ 978 6575  223]
 [ 176   65 7535]]
Epoch: 996, Train Loss: 0.628218
-----------------------------------------
Valid Accuracy: 91.61%
Confusion Matrix:
[[88561  5404  2517]
 [  355  2306    88]
 [   24     0   745]]

Precision: 0.9706408184562998
Recall: 0.91612
F1 Score: 0.9365357525561068
20 92.743
-----------------------------------------


Train Accuracy: 92.26%
Confusion Matrix:
[[7412  199  165]
 [ 975 6575  226]
 [ 181   60 7535]]
Epoch: 997, Train Loss: 0.628264
-----------------------------------------
Valid Accuracy: 9

-----------------------------------------
Valid Accuracy: 91.28%
Confusion Matrix:
[[88225  5777  2480]
 [  361  2306    82]
 [    9    15   745]]

Precision: 0.970351113858023
Recall: 0.91276
F1 Score: 0.9343463109875996
39 92.743
-----------------------------------------


Train Accuracy: 92.28%
Confusion Matrix:
[[7417  194  165]
 [ 977 6575  224]
 [ 181   60 7535]]
Epoch: 1016, Train Loss: 0.62832
-----------------------------------------
Valid Accuracy: 91.70%
Confusion Matrix:
[[88651  5286  2545]
 [  355  2306    88]
 [   22     2   745]]

Precision: 0.9707779749410684
Recall: 0.91702
F1 Score: 0.9371518926670881
40 92.743
-----------------------------------------


Train Accuracy: 92.28%
Confusion Matrix:
[[7416  194  166]
 [ 978 6575  223]
 [ 178   63 7535]]
Epoch: 1017, Train Loss: 0.628055
-----------------------------------------
Valid Accuracy: 91.15%
Confusion Matrix:
[[88097  5830  2555]
 [  341  2306   102]
 [    9    15   745]]

Precision: 0.9704632632157248
Recall: 0.

Train Accuracy: 92.31%
Confusion Matrix:
[[7425  187  164]
 [ 978 6575  223]
 [ 191   50 7535]]
Epoch: 1036, Train Loss: 0.627701
-----------------------------------------
Valid Accuracy: 91.46%
Confusion Matrix:
[[88409  5487  2586]
 [  341  2306   102]
 [   24     0   745]]

Precision: 0.9706563699546424
Recall: 0.9146
F1 Score: 0.9356279580649371
60 92.743
-----------------------------------------


Train Accuracy: 92.32%
Confusion Matrix:
[[7427  185  164]
 [ 975 6575  226]
 [ 184   57 7535]]
Epoch: 1037, Train Loss: 0.627593
-----------------------------------------
Valid Accuracy: 91.79%
Confusion Matrix:
[[88741  5192  2549]
 [  341  2306   102]
 [   24     0   745]]

Precision: 0.9710093735891624
Recall: 0.91792
F1 Score: 0.9378030024922774
61 92.743
-----------------------------------------


Train Accuracy: 92.32%
Confusion Matrix:
[[7426  187  163]
 [ 980 6575  221]
 [ 187   54 7535]]
Epoch: 1038, Train Loss: 0.627687
-----------------------------------------
Valid Accuracy:

-----------------------------------------
Valid Accuracy: 91.56%
Confusion Matrix:
[[88505  5511  2466]
 [  341  2306   102]
 [   13    11   745]]

Precision: 0.9708036771035293
Recall: 0.91556
F1 Score: 0.9362454810391461
80 92.743
-----------------------------------------


Train Accuracy: 92.33%
Confusion Matrix:
[[7429  184  163]
 [ 977 6575  224]
 [ 184   57 7535]]
Epoch: 1057, Train Loss: 0.627562
-----------------------------------------
Valid Accuracy: 91.31%
Confusion Matrix:
[[88263  5749  2470]
 [  341  2306   102]
 [   10    14   745]]

Precision: 0.9705817592357272
Recall: 0.91314
F1 Score: 0.9346724955268676
81 92.743
-----------------------------------------


Train Accuracy: 92.33%
Confusion Matrix:
[[7429  186  161]
 [ 979 6575  222]
 [ 192   49 7535]]
Epoch: 1058, Train Loss: 0.627516
-----------------------------------------
Valid Accuracy: 91.30%
Confusion Matrix:
[[88249  5674  2559]
 [  341  2306   102]
 [   10    14   745]]

Precision: 0.9706097310389515
Recall: 

<All keys matched successfully>

In [20]:
Cf.load_state_dict(torch.load('cnn30.pt'))

<All keys matched successfully>

In [21]:
valid_latents = []
VR,VL = [], []
valid_loss = 0.0
for j, (inp, lab) in enumerate(train_loader):
    probability = 0.0
    coin_toss = random.random()  

    if coin_toss < probability:
        continue
    inp = inp.to(device)
    lab = lab.to(device)

    with torch.no_grad():
        # Forward pass
        outp = Cf(inp)

    VR.append(outp)
    VL.append(lab)

VR = torch.cat(VR, dim=0)
VL = torch.cat(VL, dim=0)
_, vpre = torch.max(VR.data, 1)
_, vlbl = torch.max(VL.data, 1)
total = vlbl.size(0)
correct = (vpre == vlbl).sum().item()

vaccuracy = 100 * correct / total
print('-----------------------------------------')
print(f'Train Accuracy: {vaccuracy:.2f}%')
lb = vlbl.cpu().detach().numpy()
pre = vpre.cpu().detach().numpy()
conf_matrix = confusion_matrix(lb,pre)
precision = precision_score(lb, pre,average=avg)
recall = recall_score(lb, pre,average=avg)
f1 = f1_score(lb, pre,average=avg)
print('Confusion Matrix:')
print(conf_matrix)
print()
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print(epochs_without_improvement, best_loss)
print('-----------------------------------------')
print()
print()

valid_latents = []
VR,VL = [], []
valid_loss = 0.0
for j, (inp, lab) in enumerate(valid_loader):
    probability = 0.0
    coin_toss = random.random()  

    if coin_toss < probability:
        continue
    inp = inp.to(device)
    lab = lab.to(device)

    with torch.no_grad():
        # Forward pass
        outp = Cf(inp)

    VR.append(outp)
    VL.append(lab)

VR = torch.cat(VR, dim=0)
VL = torch.cat(VL, dim=0)
_, vpre = torch.max(VR.data, 1)
_, vlbl = torch.max(VL.data, 1)
total = vlbl.size(0)
correct = (vpre == vlbl).sum().item()

vaccuracy = 100 * correct / total
print('-----------------------------------------')
print(f'Valid Accuracy: {vaccuracy:.2f}%')
lb = vlbl.cpu().detach().numpy()
pre = vpre.cpu().detach().numpy()
conf_matrix = confusion_matrix(lb,pre)
precision = precision_score(lb, pre,average=avg)
recall = recall_score(lb, pre,average=avg)
f1 = f1_score(lb, pre,average=avg)
print('Confusion Matrix:')
print(conf_matrix)
print()
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print(epochs_without_improvement, best_loss)
print('-----------------------------------------')
print()
print()

valid_latents = []
VR,VL = [], []
valid_loss = 0.0
for j, (inp, lab) in enumerate(test_loader):
    probability = 0.0
    coin_toss = random.random()  

    if coin_toss < probability:
        continue
    inp = inp.to(device)
    lab = lab.to(device)

    with torch.no_grad():
        # Forward pass
        outp = Cf(inp)

    VR.append(outp)
    VL.append(lab)

VR = torch.cat(VR, dim=0)
VL = torch.cat(VL, dim=0)
_, vpre = torch.max(VR.data, 1)
_, vlbl = torch.max(VL.data, 1)
total = vlbl.size(0)
correct = (vpre == vlbl).sum().item()

vaccuracy = 100 * correct / total
print('-----------------------------------------')
print(f'Test Accuracy: {vaccuracy:.2f}%')
lb = vlbl.cpu().detach().numpy()
pre = vpre.cpu().detach().numpy()
conf_matrix = confusion_matrix(lb,pre)
precision = precision_score(lb, pre,average=avg)
recall = recall_score(lb, pre,average=avg)
f1 = f1_score(lb, pre,average=avg)
print('Confusion Matrix:')
print(conf_matrix)
print()
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print(epochs_without_improvement, best_loss)
print('-----------------------------------------')
print()
print()

-----------------------------------------
Train Accuracy: 92.23%
Confusion Matrix:
[[7406  206  164]
 [1002 6575  199]
 [ 241    0 7535]]

Precision: 0.926647986843666
Recall: 0.9223251028806584
F1 Score: 0.9222021194645675
100 92.743
-----------------------------------------


-----------------------------------------
Valid Accuracy: 92.74%
Confusion Matrix:
[[89692  4428  2362]
 [  361  2306    82]
 [   24     0   745]]

Precision: 0.9719064586579008
Recall: 0.92743
F1 Score: 0.9439778464490483
100 92.743
-----------------------------------------


-----------------------------------------
Test Accuracy: 94.24%
Confusion Matrix:
[[150824   5887   2888]
 [   553   2868     90]
 [    33      0    939]]

Precision: 0.977343440994377
Recall: 0.9424007508440901
F1 Score: 0.9556837182313904
100 92.743
-----------------------------------------




In [19]:
 # Iterate over mini-batches
latents = []
train_label = []
for i, (inputs, labels, lbl) in enumerate(train_loader):

    inputs = inputs.to(device)
    lbl = lbl.to(device)
    
    with torch.no_grad():

        # Forward pass
        latent = Encoder(inputs)


    latents.append(latent)
    train_label.append(lbl)


ValueError: not enough values to unpack (expected 3, got 2)

In [ ]:
latents = torch.cat(latents,dim=0)

In [ ]:
train_label = torch.cat(train_label,dim=0)

In [ ]:
latents = latents.cpu().detach().numpy()

In [ ]:
rs = reconstructData(latents)

In [ ]:
torch.argmax(train_label,dim=1)

In [ ]:
labl = torch.argmax(train_label,dim=1).cpu().detach().numpy()

In [ ]:
rs.shape

In [ ]:
real_data_reshaped = np.reshape(latents, (latents.shape[0], -1))


# Perform tsne on real and fake data
perplexity = min(30, real_data_reshaped.shape[0] - 1)
tsne = TSNE(n_components=2, perplexity=perplexity, random_state=seed)
real_tsne = tsne.fit_transform(real_data_reshaped)
label_names = ['No-Movement', 'Small Movement',  'Large Movement']
# Plot the t-SNE visualization with labeled data
plt.figure(figsize=(20, 15))
for label in np.unique(labl):
    indices = np.where(labl == label)
    plt.scatter(real_tsne[indices, 0], real_tsne[indices, 1], label=label_names[int(label)], edgecolors='black')
    
plt.legend()
plt.show()

In [ ]:
-----------------------------------------
Train Accuracy: 92.23%
Confusion Matrix:
[[7406  206  164]
 [1002 6575  199]
 [ 241    0 7535]]

Precision: 0.926647986843666
Recall: 0.9223251028806584
F1 Score: 0.9222021194645675
100 92.743
-----------------------------------------


-----------------------------------------
Valid Accuracy: 92.74%
Confusion Matrix:
[[89692  4428  2362]
 [  361  2306    82]
 [   24     0   745]]

Precision: 0.9719064586579008
Recall: 0.92743
F1 Score: 0.9439778464490483
100 92.743
-----------------------------------------


-----------------------------------------
Test Accuracy: 94.24%
Confusion Matrix:
[[150824   5887   2888]
 [   553   2868     90]
 [    33      0    939]]

Precision: 0.977343440994377
Recall: 0.9424007508440901
F1 Score: 0.9556837182313904
100 92.743
-----------------------------------------

In [ ]:
-----------------------------------------
Train Accuracy: 88.61%
Confusion Matrix:
[[6682  954  140]
 [ 945 6710  121]
 [ 435   62 7279]]

Precision: 0.8875690656187784
Recall: 0.8861025377229081
F1 Score: 0.8866658095578258
100 80.479
-----------------------------------------


-----------------------------------------
Valid Accuracy: 80.48%
Confusion Matrix:
[[77819 17188  1475]
 [  763  1940    46]
 [   33    16   720]]

Precision: 0.9603073569758481
Recall: 0.80479
F1 Score: 0.8661478300003829
100 80.479
-----------------------------------------


-----------------------------------------
Test Accuracy: 84.33%
Confusion Matrix:
[[135089  22386   2124]
 [  1087   2374     50]
 [    47     23    902]]

Precision: 0.9683679846076022
Recall: 0.8432673906948964
F1 Score: 0.8945902175201007
100 80.479
-----------------------------------------